In [1]:
# =============================================================================
# Import Libraries
# =============================================================================
import requests                     # For making API calls to CoinMarketCap
import pandas as pd                 # For data manipulation and analysis
import numpy as np                  # For numerical operations
import matplotlib.pyplot as plt     # For plotting graphs
import datetime                     # For handling date and time information
import tweepy                       # For accessing the Twitter API
import nltk                         # For Natural Language Processing
from nltk.sentiment.vader import SentimentIntensityAnalyzer  # VADER for sentiment analysis
from sklearn.linear_model import LinearRegression           # For predictive modeling
from sklearn.metrics import mean_absolute_error, r2_score      # For model evaluation

In [6]:
# =============================================================================
# Function to Fetch Crypto Data From CoinMarketCap API
# =============================================================================
def get_crypto_data(api_key, start=1, limit=5, convert='USD'):
    """
    Gets crypto listings form CMC API(CoinMarketCAP)

    Paraneters:
    api_key: CMC API KEY
    start: This the starting rank(the crypto we're choosing/starting from. Set to 1 for Bitcoin)
    limit: This is the amount of cryptocurrencies to retrieve
    convert: the fiat currency to convert the prices

    Returns:
       Dataframe: A panda df containing crypto data 
    """
    url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"


    parameters = {
        'start': str(start),
        'limit': str(limit),
        'convert': convert 
    }

    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key
    }

    #API Request
    response = requests.get(url, headers=headers, params=parameters)
    data = response.json()

    #Turns Crypto list to dataframe
    crypto_list = data['data']
    df = pd.DataFrame(crypto_list)

    # Convert the nested 'quote' column(dictionary) into columns
    df_quote = df['quote'].apply(lambda x: x[convert])
    df_quote = pd.json_normalize(df_quote)

    if 'last_updated' in df_quote.columns:
        df_quote = df_quote.drop(columns=['last_updated'])

    # Combine the data
    df = pd.concat([df.drop(columns=['quote']), df_quote], axis=1)

    #Convert the "last_updated" column (from quote but concated to df)
    df['last_updated'] = pd.to_datetime(df['last_updated'])
    return df
     
api_key = 'c936ec13-bf84-4335-b1df-aa936ab8d2b5'
cryp_df = get_crypto_data(api_key, start=1, limit=5)
print("Data Sample")
print(cryp_df.head())

Data Sample
     id         name symbol      slug  num_market_pairs  \
0     1      Bitcoin    BTC   bitcoin             11969   
1  1027     Ethereum    ETH  ethereum              9963   
2    52          XRP    XRP       xrp              1546   
3   825  Tether USDt   USDT    tether            118560   
4  1839          BNB    BNB       bnb              2382   

                 date_added  \
0  2010-07-13T00:00:00.000Z   
1  2015-08-07T00:00:00.000Z   
2  2013-08-04T00:00:00.000Z   
3  2015-02-25T00:00:00.000Z   
4  2017-07-25T00:00:00.000Z   

                                                tags    max_supply  \
0  [mineable, pow, sha-256, store-of-value, state...  2.100000e+07   
1  [pos, smart-contracts, ethereum-ecosystem, coi...           NaN   
2  [medium-of-exchange, enterprise-solutions, xrp...  1.000000e+11   
3  [stablecoin, asset-backed-stablecoin, ethereum...           NaN   
4  [marketplace, centralized-exchange, payments, ...           NaN   

   circulating_supply  to